In [ ]:
import os
import random
import shutil
import pandas as pd
import csv

### Batch Processing
Used to generate batches for manual grading

#### Get list of all submissions

In [ ]:
ANON_PATH = "../data/anonymised_assignments"

submissions = []

for path in os.listdir(ANON_PATH):
    if path.startswith('.'):
        continue
    for subpath in os.listdir(os.path.join(ANON_PATH, path)):
        submissions.append(os.path.join(ANON_PATH, path, subpath))

print(submissions[:10])

#### Get existing batches

In [ ]:
BATCH_MASTER_PATH = '../data/batches/all_batches.csv'

batch_list = []

with open(BATCH_MASTER_PATH) as f:
    reader = csv.reader(f)

    # Skip Header
    next(reader, None)

    for row in reader:
        batch_list.append(row[0])
print(batch_list[:10])
len(batch_list)

In [ ]:
submissions_set = set(submissions) - set(batch_list)
'../data/anonymised_assignments/18~19/18~19_Submission_12' in submissions_set

In [ ]:
random.sample(sorted(submissions_set), 4)

#### Create sampled batches

In [ ]:
SUBMISSIONS_PER_BATCH = 20
NUMBER_OF_BATCHES = 7

submissions_set = set(submissions) - set(batch_list)

batches = []

for i in range(NUMBER_OF_BATCHES):
    sample = random.sample(sorted(submissions_set), SUBMISSIONS_PER_BATCH)
    batches.append(sample)

    for p in sample:
        batch_list.append(p)

    submissions_set -= set(sample)

print(batches)

#### Copy Batches to directory

In [ ]:
BATCHES_PATH = "../data/batches/"

if not os.path.isdir(BATCHES_PATH):
    os.mkdir(BATCHES_PATH)

START_NUMBER = 8

for i in range(len(batches)):
    path = os.path.join(BATCHES_PATH, str(START_NUMBER + i))
    os.mkdir(path)

    for submission in batches[i]:
        split = submission.split('/')
        new_path = os.path.join(path, split[len(split) - 1])

        shutil.copytree(submission, new_path)
        shutil.make_archive(new_path, "zip", new_path)

#### Create CSV of "students" for Gradescope

In [ ]:
for i in range(len(batches)):
    roster = []
    for submission in batches[i]:
        submission_number = submission.split('_')[3]

        roster.append([submission_number, '', submission_number + '@study.edu'])

    roster_df = pd.DataFrame(roster)
    roster_df.columns = ['First Name', 'Last Name', 'Email']
    roster_df.to_csv(os.path.join(BATCHES_PATH, str(START_NUMBER + i), 'roster.csv'))



In [ ]:
with open(BATCH_MASTER_PATH, 'w') as f:
    writer = csv.writer(f)

    writer.writerow(['path'])

    for p in batch_list:
        writer.writerow([p])